In [16]:
cd ./drive/My Drive/學業/巨量/HW3

[Errno 2] No such file or directory: './drive/My Drive/學業/巨量/HW3'
/content/drive/My Drive/學業/巨量/HW3


In [ ]:
!apt-get -y install openjdk-8-jre-headless

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u275-b01-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 2s (12.6 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u275-b01-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u275-b01-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u275-b01-0ubunt

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 61kB/s 
     |████████████████████████████████| 204kB 39.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=e6cf1159fa7f15cf980cf9772a4f1e22f5c4a0b5607ee7eda554639aeb1434d1
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [17]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.master("local").getOrCreate()
#sc = SparkContext.getOrCreate()
#sqlContext = SQLContext(sc)

In [ ]:
from pyspark import SparkContext 
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.rdd import RDD


In [ ]:
conf = SparkConf().setAppName("HW3 Q1")
sc = SparkContext.getOrCreate(conf=conf)
text_file = sc.textFile("Youvegottofindwhatyoulove.txt")
sents = text_file.filter(lambda line: len(line)>0)
lineLength = text_file.flatMap(lambda s: s.split('. ')) \
                      .filter(lambda line:len(line)>0)
N = lineLength.count()
#N = sents.count()

In [ ]:
counts = text_file.flatMap(lambda line: line.split(' ')) \
             .filter(lambda word: len(word)>0 and word != '\x00') \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b) \
             .map(lambda x: (x[1], x[0])) \
             .sortByKey(ascending=False) \
             .map(lambda x: (x[1], x[0], x[0]/N))

In [ ]:
print("Word\tFrequencies\tOccurrence per Sentence")
print("----\t-----------\t----------------------")
for w, cnt, occur in counts.take(30):
    print("{}\t{}\t\t\t{}".format(w, cnt, occur))

Word	Frequencies	Occurrence per Sentence
----	-----------	----------------------
the	90			0.6382978723404256
I	86			0.6099290780141844
to	71			0.5035460992907801
and	49			0.3475177304964539
was	47			0.3333333333333333
a	46			0.3262411347517731
of	40			0.28368794326241137
that	38			0.2695035460992908
in	33			0.23404255319148937
is	28			0.19858156028368795
you	27			0.19148936170212766
it	27			0.19148936170212766
my	25			0.1773049645390071
had	22			0.15602836879432624
with	18			0.1276595744680851
It	18			0.1276595744680851
for	17			0.12056737588652482
have	17			0.12056737588652482
And	17			0.12056737588652482
all	16			0.11347517730496454
your	14			0.09929078014184398
out	14			0.09929078014184398
as	14			0.09929078014184398
what	14			0.09929078014184398
be	13			0.09219858156028368
from	13			0.09219858156028368
me	12			0.0851063829787234
on	12			0.0851063829787234
at	11			0.07801418439716312
so	11			0.07801418439716312


In [18]:
conf = SparkConf().setAppName("HW3 Q2")
sc = SparkContext.getOrCreate(conf=conf)

# Load DataFrame
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('yellow_tripdata_2017-09.csv')

# Show Schema
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [ ]:
df.count()

8945459

In [ ]:
df.groupby('passenger_count').count().show(15)

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              7|     27|
|              3| 379793|
|              8|     34|
|              0|  14658|
|              5| 419504|
|              6| 268073|
|              9|     35|
|              1|6361291|
|              4| 173538|
|              2|1328506|
+---------------+-------+



In [19]:
# Deal with missing values
df = df.withColumn("passenger_count", F.when((df["passenger_count"] == "0"), 1).otherwise(df["passenger_count"]))

In [20]:
df.filter((df["passenger_count"] == "0") | \
          (df["passenger_count"] == "") | \
          df["passenger_count"].isNull() | \
          F.isnan(df["passenger_count"]) | \
          (df["total_amount"] == "") | \
          df["total_amount"].isNull() | \
          F.isnan(df["total_amount"])).count()

0

In [ ]:
df = df.filter(df["payment_type"] == "1").select("passenger_count", "total_amount", "trip_distance")

df.show(5)

+---------------+------------+-------------+
|passenger_count|total_amount|trip_distance|
+---------------+------------+-------------+
|              1|         4.8|          .40|
|              2|        7.55|          .90|
|              1|        11.3|         1.50|
|              1|         8.8|          .90|
|              1|       31.56|         6.34|
+---------------+------------+-------------+
only showing top 5 rows



In [ ]:
average_amounts = df.groupby('passenger_count').agg({"total_amount": "average", "trip_distance": "average"})
average_amounts.show(15)

+---------------+------------------+------------------+
|passenger_count| avg(total_amount)|avg(trip_distance)|
+---------------+------------------+------------------+
|              7| 65.42318181818182| 6.298181818181818|
|              3|18.156634185770997|3.1448369322168106|
|              8|             67.45| 5.343461538461539|
|              5|18.214794738199902|3.2324018079526002|
|              6|18.039043013855164| 3.194390377953453|
|              9| 68.78840000000001|6.5527999999999995|
|              1| 17.95413006221656| 3.087874226428824|
|              4|18.317789057005374|3.1764583112617832|
|              2|18.773330411916394| 3.287393844707793|
+---------------+------------------+------------------+

